<a href="https://colab.research.google.com/github/erturkkadir/kurtahmini/blob/master/TransformerKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
from tensorflow  import keras
from tensorflow.keras import layers

In [ ]:
raw_data = pd.read_csv('./sample_data/kur.csv', header=None, names=["i", "t", "y"])
a_data = raw_data['y']
print(len(a_data))

In [ ]:
def to_seq(seq_size, a_data):
  x, y = [], []
  for i in range(len(a_data)-seq_size):
    window = a_data[i:(i+seq_size)]
    after_window = a_data[i+seq_size]
    window = [[x] for x in window]
    x.append(window)
    y.append(after_window)
  return np.array(x[:150]), np.array(y[:150]), np.array(x[150:]), np.array(y[150:])


In [ ]:
seq_size  = 15
x_train, y_train, x_test, y_test = to_seq(seq_size, a_data)    
print(x_train[-1])
print(y_train[-1])

print(x_test[-1])
print(y_test[-1])

In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.2):
  x = layers.LayerNormalization(epsilon=1e-6)(inputs)
  x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x,x)
  x = layers.Dropout(dropout)(x)
  res = x + inputs

  x = layers.LayerNormalization(epsilon=1e-6) (res)
  x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="tanh")(x)
  x = layers.Dropout(dropout)(x)
  x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
  return x + res

In [ ]:
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0.2, mlp_dropout=0.2):
  inputs = keras.Input(shape=input_shape)
  x = inputs
  for _ in range(num_transformer_blocks):
    x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)
  x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
  for dim in mlp_units:
    x = layers.Dense(dim, activation="relu")(x)
    x = layers.Dropout(mlp_dropout)(x)
  outputs = layers.Dense(1)(x)
  return keras.Model(inputs, outputs)


In [ ]:
print(x_train)
print(y_train)

In [ ]:
input_shape = x_train.shape[1:]
model = build_model(input_shape, head_size=256, num_heads=4, ff_dim=4, num_transformer_blocks=4, mlp_units=[128], dropout=0.2, mlp_dropout=0.2)
model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=1e-4))
callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]
model.fit(x_train, y_train, validation_split=0.2, epochs=200, batch_size=10, callbacks=callbacks)

model.evaluate(x_test, y_test, verbose=1)


In [ ]:
from sklearn import metrics
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred, y_test))
print("Score :  {}".format(score))
for i in range(seq_size-1):
  print(y_test[i], pred[i], y_test[i]-pred[i])



In [ ]:
import matplotlib.pyplot as plt
plt.plot(y_test)
plt.plot(pred)
plt.show()
